# Ray Tune Test
Ray tune is a hyperparameter tuning library. Trying to figure out how this works

## Issues
1. How do I set which GPU to use? Disabled my GPU choosing code to make this run(Uncomment that thing in model_train later!). 
2. Trials are somehow limited to only 3? And then it just stops. How to make it do more tries.
3. We can make ray tune run trials in parallel. (With fractional GPUs, more CPUs etc.) Maybe explore
4. using the tune.report() reports the results and will be used to early stop training. Currently, I only report at the very end. Which is not ideal. Modify to report at each stage!

In [25]:
from torch.optim import Adam, SGD
import torch.nn as nn
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import preprocessing
from inverse_modelling_tfo.models import train_model, create_perceptron_model, train_model_wtih_reporting
from inverse_modelling_tfo.data import generate_data_loaders, equidistance_detector_normalization, constant_detector_count_normalization
from inverse_modelling_tfo.data.intensity_interpolation import get_interpolate_fit_params_custom
from inverse_modelling_tfo.data.interpolation_function_zoo import *

In [26]:
data = pd.read_pickle(r'/home/rraiyan/personal_projects/tfo_inverse_modelling/data/intensity/intensity_summed_sim_data_equidistance_detector_extended.pkl')
equidistance_detector_normalization(data)

data = data[data["Wave Int"] == 2.0]    # Keep only 1 wavelength
data['Intensity'] = np.log10(data['Intensity'])        # Far values wayy to small to affect anything. Take log

# From long to wide
data = pd.pivot(data, index=['Wave Int', 'Uterus Thickness', 'Maternal Wall Thickness', "Maternal Mu_a", "Fetal Mu_a"], columns="SDD", values="Intensity").reset_index()
# Since SDD's have integer values, the columns get integer names - > which can't be accessed -> str conversion
data.columns = [str(X) for X in data.columns]   
x_columns = list(filter(lambda X: X.isdigit(), data.columns))

In [43]:
a = data['Maternal Mu_a'].unique()
b = data['Fetal Mu_a'].unique()

In [44]:
from itertools import product
m = product(a, b)

In [45]:
n = list(m)
len(n)

84

In [28]:
data.columns

Index(['Wave Int', 'Uterus Thickness', 'Maternal Wall Thickness',
       'Maternal Mu_a', 'Fetal Mu_a', '10', '14', '19', '23', '28', '32', '37',
       '41', '46', '50', '55', '59', '64', '68', '73', '77', '82', '86', '91',
       '95'],
      dtype='object')

In [29]:

params = {
        'batch_size': 32, 'shuffle': True, 'num_workers': 2
    }
train, val = generate_data_loaders(data, params, x_columns, y_columns, 0.8)